In [ ]:
class UnstructuredDb():
    def __init__(self,path:str="chatbot/",table:str='Psychological_counseling'):
        import chromadb
        import os
        from chromadb.db.base import UniqueConstraintError
        from chromadb.utils import embedding_functions
        
        if not os.path.exists(path):
            os.makedirs(path, exist_ok=True)
            print(f"Directory '{path}' created successfully.")

         
        
        client = chromadb.PersistentClient(path=path)
        em = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="Huffon/sentence-klue-roberta-base")
        try:
            self.collection = client.create_collection(name=table, embedding_function=em)
        except UniqueConstraintError: 
            self.collection = client.get_collection(name=table, embedding_function=em)
            
    def get(self,question:str)->[str]:
        
        results=self.collection.query(query_texts=[question],n_results=1)
        return results


    def add(self,key:str=None,question:str=None,answer:str=None):

        self.collection.add(
        documents=[question],
        metadatas=[answer],
        ids=[key]
        )

In [ ]:
db=UnstructuredDb()

In [ ]:
# import pandas as pd
# import shutil

# data=pd.read_csv('wellness_dataset.csv')
# for index, row in data.iterrows():
#     print((index/1034)*100)
#     db.add(answer={row['챗봇']:row['구분']},question=row['유저'],key=str(index))

# # 디렉토리를 ZIP 파일로 압축
# shutil.make_archive('chatbot', 'zip', './chatbot')


In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/', methods=['GET'])
def home():
    return render_template("chatbot.html")


if __name__ == '__main__':
    app.run(host="0.0.0.0", port=9999)
